In [2]:
import pandas as pd
import json
from datetime import datetime

In [4]:
#checking if function is working well or not
match_data = load_json(r'C:\Users\yoges\OneDrive\Documents\Programming\Proj-X\match.json')

In [5]:
# Function to read a json data file
def load_json(file_path):
    try:
        with open (file_path, 'r') as file:
            
            data = json.load(file)
            return data
        
    except FileNotFoundError:
        
        print(f"Error: File {file_path} not found.")
        return None
    
    except json.JSONDecodeError:
        
        print(f"Invalid Json format.")
        return None
    

 check the various types of record returned by the function

In [6]:
segment_types = set(segment['type'] for segment in match_data['data']['segments'])
segment_types

{'player-loadout',
 'player-round',
 'player-round-damage',
 'player-round-kills',
 'player-summary',
 'round-summary',
 'team-summary'}

In [27]:
df_collection = {}

In [12]:
match_data['data']['attributes']

{'id': 'd2j89095d-7fc4-4d17-8gi1-8937370377dc'}

In [20]:
match_data['data']['metadata']

{'modeKey': 'bomb',
 'modeName': 'Competitive',
 'modeImageUrl': 'https://trackercdn.com/cdn/tracker.gg/valorant/icons/modes/normal.png',
 'modeMaxRounds': 25,
 'duration': 1893354,
 'dateStarted': '2025-06-30T21:43:37.089+00:00',
 'rounds': 20,
 'isRanked': True,
 'queueId': 'competitive',
 'playlist': 'competitive',
 'map': '2fe4ed3a-450a-948b-6d6b-e89a78e680a9',
 'mapName': 'Lotus',
 'mapImageUrl': 'https://trackercdn.com/cdn/tracker.gg/valorant/db/maps/7.04/lotus.jpg'}

In [17]:
match_data['data']['segments']

[{'type': 'team-summary',
  'attributes': {'teamId': 'Red'},
  'metadata': {'name': 'Red', 'hasWon': True},
  'expiryDate': '0001-01-01T00:00:00+00:00',
  'stats': {'roundsWon': {'rank': None,
    'percentile': None,
    'displayName': None,
    'displayCategory': None,
    'category': None,
    'description': None,
    'metadata': {},
    'value': 13,
    'displayValue': '13',
    'displayType': 'Number'},
   'roundsLost': {'rank': None,
    'percentile': None,
    'displayName': None,
    'displayCategory': None,
    'category': None,
    'description': None,
    'metadata': {},
    'value': 7,
    'displayValue': '7',
    'displayType': 'Number'},
   'score': {'rank': None,
    'percentile': None,
    'displayName': None,
    'displayCategory': None,
    'category': None,
    'description': None,
    'metadata': {},
    'value': 21931,
    'displayValue': '21,931',
    'displayType': 'Number'},
   'kills': {'rank': None,
    'percentile': None,
    'displayName': None,
    'displayC

In [22]:
match_id = match_data['data']['attributes']['id']
match_duration = match_data['data']['metadata']['duration']
match_date = match_data['data']['metadata']['dateStarted']
match_mode = match_data['data']['metadata']['modeName']
match_rounds = match_data['data']['metadata']['rounds']
match_mapId = match_data['data']['metadata']['map']
match_map = match_data['data']['metadata']['mapName']


In [23]:
match_summary_df = pd.DataFrame(
    {
        'Match_ID' : [match_id],
        'Duration': [match_duration],
        'Date' : [match_date],
        'Mode' : [match_mode],
        'Rounds' : [match_rounds],
        'Map_ID' : [match_mapId],
        'Map' : [match_map],
        
    }
)

In [25]:
match_summary_df

,Match_ID,Duration,Date,Mode,Rounds,Map_ID,Map
0,d2j89095d-7fc4-4d17-8gi1-8937370377dc,1893354,2025-06-30T21:43:37.089+00:00,Competitive,20,2fe4ed3a-450a-948b-6d6b-e89a78e680a9,Lotus


In [30]:
df_collection['match_summary_df'] = match_summary_df
df_collection

{'match_summary_df':                                 Match_ID  Duration  \
 0  d2j89095d-7fc4-4d17-8gi1-8937370377dc   1893354   
 
                             Date         Mode  Rounds  \
 0  2025-06-30T21:43:37.089+00:00  Competitive      20   
 
                                  Map_ID    Map  
 0  2fe4ed3a-450a-948b-6d6b-e89a78e680a9  Lotus  }

Team Summary

In [34]:
segments = match_data['data']['segments']

In [36]:
team_summary = []

for segment in segments:
    if segment['type'] == 'team-summary':
        team_summary.append({
            'team_id': segment['attributes']['teamId'],
            'name': segment['metadata']['name'],
            'has_won': segment['metadata']['hasWon'],
            'rounds_won': segment['stats']['roundsWon']['value'],
            'rounds_lost': segment['stats']['roundsLost']['value'],
            'score': segment['stats']['score']['value'],
            'kills': segment['stats']['kills']['value'],
            'assists': segment['stats']['assists']['value'],
            'damage': segment['stats']['damage']['value'],
        })

In [37]:
team_df = pd.DataFrame(team_summary)
team_df

,team_id,name,has_won,rounds_won,rounds_lost,score,kills,assists,damage
0,Red,Red,True,13,7,21931,81,33,14584
1,Blue,Blue,False,7,13,18904,66,17,12365


In [ ]:
# as data is more complex to read we need to make a function to process this data

def process_match_data(data):
    if not data or 'data' not in data:
        return None, None, None
    
    match_data = data['data']
    metadata = match_data['metadata']
    segments = match_data['segments']
    
    #match summary
    
    match_summary = {
        'match_id': match_data['attributes']['id'],
        'map_name':metadata['mapName'],
        'mode': metadata['modeName'],
        'date_started': datetime.strptime(metadata['dateStarted'], '%Y-%m-%d%H:%M:%S.%f%z'),
        'duration_minutes': metadata['duration']/1000/60, #here converting ms to secounds
        'rounds_played': metadata['rounds'],
        'is_ranked': metadata['isRanked']
    }
    match_df = pd.DataFrame([match_summary])
    
    #Creating a team summary DataFrame
    
    team_data = []
    for segment in segments:
        if segment['type'] == 'team-summary':
            team_data.append({
                'team_id': segment['attributes']['teamId'],
                'name': segment['metadata']['name'],
                'has_won': segment['metadata']['hasWon'],
                'rounds_won': segment['stats']['roundsWon']['value'],
                'rounds_lost': segment['stats']['roundsLost']['value'],
                'score': segment['stats']['score']['value'],
                'kills': segment['stats']['kills']['value'],
                'deaths': segment['stats']['deaths']['value'],
                'assists': segment['stats']['assists']['value'],
                'damage': segment['stats']['damage']['value']
            })
    team_df = pd.DataFrame(team_data)
    
    